In [1]:
from datetime import datetime
from pathlib import Path
import pandas as pd
import tushare as ts
import numpy as np
import random as rd
import time,sys,os,sqlite3
import traceback

# 用来屏蔽方法内print
class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout
        
# with HiddenPrints():
#     print("111111")
# print("222222")

# 快速连接数据库
class fast_db_conn:
    def __init__(self,db_path):
        self._db = db_path
    def __enter__(self):
        self._conn = sqlite3.connect(self._db)
        print("打开连接 {}".format(self._db))
        return self._conn
    def __exit__(self, exc_type, exc_val, exc_tb):
        self._conn.commit()
        self._conn.close()
        print("关闭连接 {}".format(self._db))
        
# with fsat_db_conn(dbpath.joinpath(dbname)) as c:

# def cal_ma(df,seq):
#     """
#     用收盘价计算n天ma，在df上直接添加相应列
#     df中需要有日期列
#     """
#     df.sort_values(by='date', ascending=True, inplace=True)
#     for i in seq:
#         df["avg"+str(i)] = df.close.rolling(i).mean().round(3)
#         df["v_avg"+str(i)] = df.volume.rolling(i).mean().round(2)
#     res = df.dropna(inplace=False).sort_values(by='date', ascending=False, inplace=False)
#     return res

# def acc_check(df,seq):
#     for i in seq:
#         if sum(df["v_avg"+str(i)] - df["v_ma"+str(i)].astype(float) > 0.001): print("voluma ma {} 计算有问题".format(i)) 
#         if sum(df["avg"+str(i)] - df["ma"+str(i)].astype(float)  > 0.001): print("voluma ma {} 计算有问题".format(i)) 
#     return 

In [31]:
# path
dbpath = Path(r'./../../../OneDrive/sqlite_db/finance.db').resolve()
# dbpath = r'D:\OneDrive\sqlite_db\finance.db'   # 笔记本
# dbpath = r'E:\OneDrive\sqlite_db\finance.db'  # 台式

datapath = Path(r'./../../data/tushare_old')
dbpath

WindowsPath('D:/OneDrive/sqlite_db/finance.db')

In [9]:
# 读取股票列表  目前手动从聚宽的数据粘贴下来的
stock_pool = pd.read_excel(str(datapath.resolve())+'/allstock_221020.xlsx',dtype = 'str')
stock_list = stock_pool[['code','display_name']]
stock_list.index = stock_list['code']
stock_list = stock_list['display_name']
stock_dict = stock_list.to_dict()  

In [29]:
# 创建本次导入数据的表
start_date = '2022-11-03'
end_date = '2022-11-03'
table_name = 'stock_all_{}_{}'.format(start_date,end_date)

ddl_path = Path(r'./sql/stock_all.sql').resolve()

with open(ddl_path,'r',encoding='utf-8') as tt:
    ddl_sql = tt.read().format(table_name)

with fast_db_conn(dbpath) as ddl_c: ddl_c.executescript(ddl_sql)
print('{} 已创建'.format(table_name))    


打开连接 D:\OneDrive\sqlite_db\finance.db
关闭连接 D:\OneDrive\sqlite_db\finance.db
stock_all_2022-11-03_2022-11-03 已创建


In [30]:
# 读取ts端口写db
t1,t3 = [],[]
with fast_db_conn(dbpath) as c:
    for i,j in enumerate(stock_dict):
#         i:index数字 j:code
#         用来计算耗时
        start = datetime.now()
        with HiddenPrints(): temp = ts.get_hist_data(j,start=start_date,end=end_date)
        after_ts = datetime.now()
        temp["code"] = j
        temp["name"] = stock_dict[j]
        temp["pk"] = temp["code"] + '_' + temp.index
#         temp = cal_ma(temp,[3,5,10,20,60])
#         acc_check(temp,[5,10,20])
#         temp["ma3"] = temp["avg3"]
#         temp["v_ma3"] = temp["v_avg3"]
#         temp["ma60"] = temp["avg60"]
#         temp["v_ma60"] = temp["v_avg60"]
        temp.to_sql(name=table_name, con=c, if_exists='append', index=True)
        after_insert = datetime.now()
        
        t1.append((after_ts - start).total_seconds())
        t3.append((after_insert - after_ts).total_seconds())
        
        if not ((i+1) % 200): 
            c.commit()
            print("完成 {} commit, 两阶段平均耗时为 {} {}".format(i+1,np.array(t1).mean().round(2),np.array(t3).mean().round(2)))
            t1,t3 = [],[]
print("完成全部{}".format(i+1))        
        


打开连接 D:\OneDrive\sqlite_db\finance.db
完成 100 commit, 两阶段平均耗时为 0.26 0.02
完成 200 commit, 两阶段平均耗时为 0.26 0.01
完成 300 commit, 两阶段平均耗时为 0.26 0.02
完成 400 commit, 两阶段平均耗时为 0.26 0.02
完成 500 commit, 两阶段平均耗时为 0.21 0.01
完成 600 commit, 两阶段平均耗时为 0.26 0.02
完成 700 commit, 两阶段平均耗时为 0.26 0.02
完成 800 commit, 两阶段平均耗时为 0.26 0.02
完成 900 commit, 两阶段平均耗时为 0.26 0.02
完成 1000 commit, 两阶段平均耗时为 0.26 0.02
完成 1100 commit, 两阶段平均耗时为 0.26 0.01
完成 1200 commit, 两阶段平均耗时为 0.26 0.01
完成 1300 commit, 两阶段平均耗时为 0.27 0.02
完成 1400 commit, 两阶段平均耗时为 0.26 0.02
完成 1500 commit, 两阶段平均耗时为 0.24 0.02
完成 1600 commit, 两阶段平均耗时为 0.26 0.02
完成 1700 commit, 两阶段平均耗时为 0.26 0.02
完成 1800 commit, 两阶段平均耗时为 0.26 0.02
完成 1900 commit, 两阶段平均耗时为 0.26 0.02
完成 2000 commit, 两阶段平均耗时为 0.26 0.02
完成 2100 commit, 两阶段平均耗时为 0.26 0.02
完成 2200 commit, 两阶段平均耗时为 0.27 0.02
完成 2300 commit, 两阶段平均耗时为 0.26 0.0
完成 2400 commit, 两阶段平均耗时为 0.22 0.0
完成 2500 commit, 两阶段平均耗时为 0.21 0.0
完成 2600 commit, 两阶段平均耗时为 0.18 0.0
完成 2700 commit, 两阶段平均耗时为 0.14 0.0
完成 2800 commit, 两阶段平均耗时为 0.26 0

In [33]:
# # 数据写入db
# dirpath = datapath.joinpath(r"./all")
# with fast_db_conn(dbpath.joinpath(dbname)) as c:
#     try:
#         for i,j in enumerate(os.listdir(dirpath)):
#             temp = pd.read_excel(str(dirpath.resolve())+r"/"+j,dtype = 'str')
# #             with HiddenPrints(): dd = ts.get_hist_data(code,start='2021-01-01',end='2022-10-21',pause=1)
#             temp["code"] = j[:6]
#             temp["name"] = stock_dict[j[:6]]
#             temp.to_sql(name="stock_all_221021", con=c, if_exists='append', index=False)
#             if not ((i+1) % 100): 
#                 c.commit()
#                 print("完成 {} commit".format(i+1))
#     except Exception as e:
#         print(e)
#         print("出错i = {} ".format(i))
#     finally: 
#         c.commit()
# print("完成全部{}".format(i+1))        



# for i,code in enumerate(clist):
#     with HiddenPrints(): dd = ts.get_hist_data(code,start='2021-06-01',end='2022-10-20')

In [ ]:
# # 获取沪深300股票
# for i,code in enumerate(hs300):
#     dd = ts.get_hist_data(code,start='2021-06-01',end='2022-10-20')
#     dd.to_excel(p+code+".xlsx")
#     print("完成{}个,代码{}".format(i+1,code))
#     time.sleep(rd.random())

In [ ]:
# 获取所有股票
for i,code in enumerate(clist):
    with HiddenPrints(): dd = ts.get_hist_data(code,start='2021-06-01',end='2022-10-20')
    dd.to_excel(p+code+".xlsx")
    if not ((i+1) % 100) :print("完成{}个,代码{}".format(i+1,code))

In [104]:
ts.get_h_data('002337',start='2021-06-01',end='2022-10-21')

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
[Getting data:]#

""


In [ ]:
def db_ingest(dict, db_path, if_tab_exists='append', event_dt=True, ingest_tm=True, index=False):
    try:
        tab_suffix = '-' + dt.datetime.now().strftime("%y%m%d")
        conn = sqlite3.connect(db_path)
        for name, df in dict.items():
            if event_dt: df['event_dt'] = dt.datetime.now().strftime("%Y-%m-%d")  # 目前为数据摄入时间，后续需要优化为自动计算上一交易日
            if ingest_tm: df['ingest_tm'] = dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # 数据写入时间
            df.to_sql(name=name + tab_suffix, con=conn, if_exists=if_tab_exists, index=index)
            print("表" + name + "写入成功")
        conn.commit()
        print("事务提交，共提交" + str(len(dict)) + "张表")
        conn.close()
        print("数据库关闭,入库成功")
    except Exception as e:
        print(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + "数据入库异常:{0}".format(e))
        traceback.print_exc()

In [68]:
dir = r"E:/gitee/stockdata/tushare_old/all/"
dd = pd.read_excel(io=dir+"000001.xlsx",dtype='str')

In [32]:
conn = sqlite3.connect("E:/gitee/finance/db/finance.db")
try:
    for i,j in enumerate(all_path):
        data = pd.read_excel(io=dir+j,dtype='str')
        data["code"] = j[:6]
        data.to_sql(name="stock_all_221020", con=conn, if_exists='append', index=False)
        if not ((i+1) % 50): 
            conn.commit()
            print("完成 {} commit".format(i+1))
    conn.commit()
except Exception as e:
    print("出错i = {} ".format(i))
finally: 
    conn.commit()
    conn.close()
print("完成全部{}".format(i+1))




In [123]:
temp = ts.get_hist_data('000001',start='2021-01-01',end='2022-10-21',pause=1)
temp

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
date,,,,,,,,,,,,,,
2022-10-21,11.21,11.25,11.09,11.07,785574.31,-0.11,-0.98,11.308,11.396,11.792,892629.41,835273.07,783041.87,0.40
2022-10-20,11.23,11.31,11.20,11.16,781923.19,-0.09,-0.80,11.396,11.471,11.866,954726.87,810438.66,811635.55,0.40
2022-10-19,11.42,11.49,11.29,11.28,923989.69,-0.19,-1.66,11.424,11.537,11.956,968865.44,824018.14,864590.28,0.48
2022-10-18,11.55,11.64,11.48,11.46,946837.12,0.00,0.00,11.486,11.619,12.028,895981.99,809711.74,861292.59,0.49
2022-10-17,11.42,11.52,11.48,11.34,1024822.75,-0.05,-0.43,11.486,11.686,12.101,789665.24,793990.26,900085.23,0.53
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-08,19.90,20.10,19.85,19.31,1195473.25,-0.05,-0.25,19.216,19.068,18.779,1618034.45,1280368.00,1053446.97,0.62
2021-01-07,19.52,19.98,19.90,19.23,1584185.25,0.34,1.74,19.114,18.909,18.719,1563840.49,1224115.69,1024066.64,0.82
2021-01-06,18.08,19.56,19.56,18.00,1934945.12,1.39,7.65,18.974,18.719,18.660,1442702.99,1140705.44,978718.43,1.00


In [111]:
temp

In [29]:
hs300 = ["688981"
,"688599"
,"688561"
,"688396"
,"688363"
,"688169"
,"688126"
,"688111"
,"688065"
,"688036"
,"688012"
,"688008"
,"688005"
,"605499"
,"603993"
,"603986"
,"603899"
,"603882"
,"603833"
,"603806"
,"603799"
,"603659"
,"603501"
,"603486"
,"603392"
,"603369"
,"603290"
,"603288"
,"603260"
,"603259"
,"603195"
,"603185"
,"603160"
,"603087"
,"603019"
,"601998"
,"601995"
,"601989"
,"601988"
,"601985"
,"601966"
,"601939"
,"601919"
,"601916"
,"601901"
,"601899"
,"601898"
,"601888"
,"601881"
,"601878"
,"601877"
,"601868"
,"601865"
,"601857"
,"601838"
,"601825"
,"601818"
,"601816"
,"601808"
,"601800"
,"601799"
,"601788"
,"601766"
,"601728"
,"601698"
,"601688"
,"601669"
,"601668"
,"601658"
,"601633"
,"601628"
,"601618"
,"601601"
,"601600"
,"601398"
,"601390"
,"601377"
,"601360"
,"601336"
,"601328"
,"601319"
,"601318"
,"601288"
,"601238"
,"601236"
,"601229"
,"601225"
,"601216"
,"601211"
,"601186"
,"601169"
,"601166"
,"601155"
,"601138"
,"601117"
,"601111"
,"601100"
,"601088"
,"601066"
,"601021"
,"601012"
,"601009"
,"601006"
,"600999"
,"600989"
,"600958"
,"600941"
,"600926"
,"600919"
,"600918"
,"600905"
,"600900"
,"600893"
,"600887"
,"600886"
,"600845"
,"600837"
,"600809"
,"600795"
,"600763"
,"600760"
,"600745"
,"600741"
,"600690"
,"600660"
,"600655"
,"600606"
,"600600"
,"600588"
,"600585"
,"600584"
,"600570"
,"600547"
,"600519"
,"600489"
,"600460"
,"600438"
,"600436"
,"600426"
,"600406"
,"600383"
,"600362"
,"600352"
,"600346"
,"600332"
,"600309"
,"600276"
,"600219"
,"600196"
,"600188"
,"600183"
,"600176"
,"600161"
,"600150"
,"600132"
,"600115"
,"600111"
,"600104"
,"600089"
,"600085"
,"600061"
,"600050"
,"600048"
,"600036"
,"600031"
,"600030"
,"600029"
,"600028"
,"600025"
,"600019"
,"600018"
,"600016"
,"600015"
,"600011"
,"600010"
,"600009"
,"600000"
,"300999"
,"300979"
,"300957"
,"300919"
,"300896"
,"300866"
,"300782"
,"300763"
,"300760"
,"300759"
,"300751"
,"300750"
,"300661"
,"300628"
,"300601"
,"300595"
,"300529"
,"300498"
,"300496"
,"300454"
,"300450"
,"300433"
,"300413"
,"300408"
,"300347"
,"300316"
,"300274"
,"300223"
,"300207"
,"300142"
,"300124"
,"300122"
,"300059"
,"300033"
,"300015"
,"300014"
,"300003"
,"003816"
,"002938"
,"002920"
,"002916"
,"002841"
,"002821"
,"002812"
,"002791"
,"002736"
,"002714"
,"002709"
,"002648"
,"002607"
,"002602"
,"002601"
,"002600"
,"002594"
,"002568"
,"002555"
,"002493"
,"002475"
,"002466"
,"002460"
,"002459"
,"002415"
,"002414"
,"002410"
,"002371"
,"002352"
,"002311"
,"002304"
,"002271"
,"002252"
,"002241"
,"002236"
,"002230"
,"002202"
,"002179"
,"002142"
,"002129"
,"002120"
,"002074"
,"002064"
,"002050"
,"002049"
,"002032"
,"002027"
,"002008"
,"002007"
,"002001"
,"001979"
,"001289"
,"000977"
,"000963"
,"000938"
,"000895"
,"000877"
,"000876"
,"000858"
,"000800"
,"000792"
,"000786"
,"000776"
,"000768"
,"000725"
,"000708"
,"000703"
,"000661"
,"000651"
,"000625"
,"000596"
,"000568"
,"000538"
,"000425"
,"000408"
,"000338"
,"000333"
,"000301"
,"000166"
,"000157"
,"000100"
,"000069"
,"000066"
,"000063"
,"000002"
,"000001"]

In [17]:
a= ts.get_hist_data('000001',start='2022-11-01',end='2022-11-03')

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


In [18]:
a

,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
date,,,,,,,,,,,,,,
2022-11-03,10.54,10.57,10.44,10.38,983535.62,-0.19,-1.79,10.500,10.620,11.046,1141760.11,1129595.83,970017.24,0.51
2022-11-02,10.61,10.68,10.63,10.48,1302988.12,-0.04,-0.38,10.548,10.696,11.117,1136529.67,1109434.59,966726.37,0.67
2022-11-01,10.38,10.68,10.67,10.36,1381234.12,0.33,3.19,10.554,10.762,11.191,1104698.03,1071534.75,940623.24,0.71


date
2022-11-03    2022-11-03_10.44
2022-11-02    2022-11-02_10.63
2022-11-01    2022-11-01_10.67
dtype: object